In [2]:
pip install -qU "langchain[google-genai]"

Note: you may need to restart the kernel to use updated packages.


In [3]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [4]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Arka")])

AIMessage(content="Hi Arka! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--f18c7128-82a1-4bdd-9035-a351caee9844-0', usage_metadata={'input_tokens': 6, 'output_tokens': 19, 'total_tokens': 25, 'input_token_details': {'cache_read': 0}})

In [8]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="As a large language model, I have no memory of past conversations. Therefore, I don't know your name. You haven't told me!", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--25378a21-d4f0-47b3-a285-655c2a282796-0', usage_metadata={'input_tokens': 6, 'output_tokens': 32, 'total_tokens': 38, 'input_token_details': {'cache_read': 0}})

In [9]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Arka"),
        AIMessage(content="Hello Arka! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Arka. You told me that at the beginning of our conversation.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--17f88789-553c-4bec-8a6b-596b969aef11-0', usage_metadata={'input_tokens': 22, 'output_tokens': 17, 'total_tokens': 39, 'input_token_details': {'cache_read': 0}})

In [5]:
pip install langchain-core langgraph>0.2.27

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [15]:
config = {"configurable": {"thread_id": "abc123"}}

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [8]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [9]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [10]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [31]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [11]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Arka, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  
        print(chunk.content, end="|")

Hi| Arka!| Here's a joke for you:

Why did the scarecrow win an| award?

Because he was outstanding in his field!
|

Till now I only had been forming normal chatbot, from next code I start using API and Search for results

In [13]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [14]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

Note: you may need to restart the kernel to use updated packages.


Getting the Weather Forecast using API call

In [17]:
import requests

def get_weather(city_name, api_key):
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    
    params = {
        'q': city_name,
        'appid': api_key,
        'units': 'metric' 
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        weather = {
            'city': data['name'],
            'temperature': data['main']['temp'],
            'weather': data['weather'][0]['description'],
            'humidity': data['main']['humidity'],
            'wind_speed': data['wind']['speed']
        }
        return weather
    else:
        return {'error': f"Failed to get weather data: {response.status_code}"}

api_key = "24b0657c347a3437a4e07a8f8030b170"
city = "Kolkata"

weather_info = get_weather(city, api_key)

print(weather_info)


{'city': 'Kolkata', 'temperature': 35.97, 'weather': 'haze', 'humidity': 56, 'wind_speed': 4.12}


Getting trending information for various city using Tavily Search

In [15]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is trending in Tokyo ?")
print(search_results)
tools = [search]

[{'title': 'Trending In Tokyo: TikTok Made Me Go', 'url': 'https://savvytokyo.com/trending-in-tokyo-tiktok-made-me-go/', 'content': 'Trending In Tokyo: TikTok Made Me Go · Oi Racecourse Flea Market · Shimokitazawa · Shibuya Sky · Shodai (Soba Noodle Shop) · BookOff Super', 'score': 0.7295395}, {'title': 'Trending in Tokyo - YouTube', 'url': 'https://www.youtube.com/playlist?list=PLKr_Y768pGvDA3wx-nMhCapTkD6SheNw4', 'content': 'Trending in Tokyo · Japanese Love Shrines and Fortunes | Hikawa Shrine in Japan Kawagoe 川越 ❤️ · Sanrio PuroLand Little Twin Stars Christmas | Vlogmas 2019.', 'score': 0.66959417}]


In [ ]:
search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is trending in Kolkata ?")
print(search_results)
tools = [search]

[{'title': 'Kolkata Trends (@kolkatatrends) • Instagram photos and videos', 'url': 'https://www.instagram.com/kolkatatrends/', 'content': "Kolkata Trends is your one-stop fashion destination for trendy men's and women's apparel, stylish accessories, and more 🤩 Whether you're looking to elevate", 'score': 0.7474137}, {'title': 'Kolkata, India X (Twitter) Trends for last 24 hours - trends24', 'url': 'https://trends24.in/india/kolkata/', 'content': "Today's top X (Twitter) trends and hashtags in Kolkata, India: #HappyBirthdayNTR, मनीष कश्यप, PMCH, #LSGvSRH, Digvesh Rathi. Explore more locations and", 'score': 0.7418428}]


In [16]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [18]:
model_with_tools = model.bind_tools(tools)

In [20]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in Guwahati?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Guwahati'}, 'id': 'f4941b7c-151d-439b-bed4-fc76a56132af', 'type': 'tool_call'}]


Some more examples for Weather Details

In [21]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [24]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in Howrah?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather in Howrah?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (9eaff211-e36a-47b6-be5f-d830a040e79d)
 Call ID: 9eaff211-e36a-47b6-be5f-d830a040e79d
  Args:
    query: weather in Howrah
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Howrah Weather Forecast 19 May 2025 - Times of India", "url": "https://timesofindia.indiatimes.com/weather/howrah-weather-forecast-today/711102", "content": "Today's Weather in Howrah: In Howrah today, the weather is expected to be Haze with a maximum temperature of 34°C and a minimum of 28°C. Sunrise in Howrah", "score": 0.95508945}, {"title": "Howrah (Uluberia) - Local Weather Report & Forecast", "url": "https://city.imd.gov.in/citywx/city_weather.php?id=42805", "content": "Local Weather Report and Forecast Fo

In [25]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The weather in San| Francisco on Monday, May 19, 2025 is sunny| with a temperature of 62.1°F (16.7|°C). The wind is WNW at 2.5 mph. The forecast for the day is sunny with a high of 64°F and a low of| 52°F. There is 0% chance of precipitation.|